### Testes do classificador usando dados em CSV

In [1]:
import pandas as pd

In [12]:
df_operacao = pd.read_csv("dados_operacao.csv")
df_metanol = pd.read_csv("dados_metanol.csv")

In [13]:
df_operacao.columns, df_metanol.columns

(Index(['id', 'Data de Ocorrência', 'Monitoramento', 'Servico', 'Usuário',
        'Conteúdo', 'Link', 'Tipo Conteudo', 'comentarios', 'curtidas',
        'manifestacoes_quantidade', 'alcance', 'alcance.1', 'amplificacao',
        'Idioma', 'termos'],
       dtype='object'),
 Index(['_id', 'data', 'monitoramento_nome.keyword', 'servico.keyword',
        'publicador_nome', 'conteudo', 'link', 'tipoconteudo', 'busca_nome',
        'manifestacoes.valor', 'curtidas', 'comentarios', 'amplificacao',
        'alcance'],
       dtype='object'))

In [14]:
df_operacao = df_operacao[["Conteúdo", "Link"]].rename(columns={"Conteúdo": "conteudo", "Link": "link"})
df_metanol = df_metanol[["conteudo", "link"]]

In [5]:
df_operacao.columns, df_metanol.columns

(Index(['conteudo', 'link'], dtype='object'),
 Index(['conteudo', 'link'], dtype='object'))

In [ ]:
df = df_operacao + df_metanol
df.head(10)

### Usando a API diretamente

In [16]:
api_url = "http://localhost:8000/api/enqueue"

In [35]:
import requests

LIMITE = 1000

for i, tweet in df.iterrows():
    if i >= LIMITE:
        print(f"Foram carregadas {i} mensagens.")
        break
    try:
        requests.post(url=api_url, json={"msg":tweet.conteudo})
    except Exception as e:
        print("Erro ao enviar mensagem %d: %s", i, str(e))

Foram carregadas 1000 mensagens.


### Usando o classificador separadamente

In [ ]:
from app.processor.classifier import BERTClassifier
from app.utils.logging_config import get_logger
import asyncio

logger = get_logger(__name__)


async def run_classifier(duration=10):
    try:
        bert = BERTClassifier()
        await bert.initialize()
        
        logger.info("Classificador iniciado.")
        
        async def auto_stop():
            await asyncio.sleep(duration)
            await bert.stop()
            logger.info("Classificador parou.")

        await asyncio.gather(
            bert.start_consuming(),
            auto_stop()
        )
    except Exception as e:
        logger.exception("Failed to initialize BERTClassifier: %s", e)

await run_classifier(duration=60)
    